In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
NETWORK = 'InceptionV3'
# NETWORK = 'Resnet_FC'
# NETWORK = 'Resnet'
# NETWORK = 'Resnet_res2c'
# NETWORK = 'Resnet_res3d'
# NETWORK = 'Resnet_res4f'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'EfficientNetV2M'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ResNet152V2'
# NETWORK = 'ConvNeXtBase'

METHOD = "MAX-AE"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'

PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"ProcessedFeatures_{METHOD}")

processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,CHUM-001,4.485706,-1.089238,7.262412,-8.944846,15.662836,-11.749378,9.908555,-8.122204,-5.735908,...,-9.915890,11.643356,-7.214621,10.840294,-11.032719,11.850606,5.513536,-13.309430,10.613845,-10.545938
1,CHUM-002,-0.871721,-2.756554,3.061361,-4.119829,13.619308,-5.428620,3.538110,-1.755272,-1.232317,...,-8.401873,5.114008,-8.425580,11.245088,-5.558995,5.322546,0.983876,-5.255292,5.934350,-5.383061
2,CHUM-006,-6.286274,-1.969023,0.316089,4.045898,26.725820,3.302373,2.846397,3.988780,5.453089,...,-1.251950,1.743170,-12.878074,12.605976,-0.240946,0.288653,-8.028224,-5.678471,1.700145,-1.731813
3,CHUM-007,-16.278902,-10.682121,-11.442419,15.026470,26.699356,11.757220,-10.014070,13.172195,14.324217,...,-8.697674,-12.457297,-16.455168,10.636695,12.132242,-11.867635,-16.044884,8.140953,-10.125163,9.394515
4,CHUM-008,-0.352161,-0.641335,3.311094,-4.010529,19.108816,-5.374633,5.588564,-3.142635,-0.881599,...,-3.396833,4.848978,-8.309546,13.379717,-5.134465,6.119023,1.801414,-7.728262,5.466892,-7.261157


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,CHUM-001,4.485706,-1.089238,7.262412,-8.944846,15.662836,-11.749378,9.908555,-8.122204,-5.735908,...,-9.915890,11.643356,-7.214621,10.840294,-11.032719,11.850606,5.513536,-13.309430,10.613845,-10.545938
1,CHUM-002,-0.871721,-2.756554,3.061361,-4.119829,13.619308,-5.428620,3.538110,-1.755272,-1.232317,...,-8.401873,5.114008,-8.425580,11.245088,-5.558995,5.322546,0.983876,-5.255292,5.934350,-5.383061
2,CHUM-006,-6.286274,-1.969023,0.316089,4.045898,26.725820,3.302373,2.846397,3.988780,5.453089,...,-1.251950,1.743170,-12.878074,12.605976,-0.240946,0.288653,-8.028224,-5.678471,1.700145,-1.731813
3,CHUM-007,-16.278902,-10.682121,-11.442419,15.026470,26.699356,11.757220,-10.014070,13.172195,14.324217,...,-8.697674,-12.457297,-16.455168,10.636695,12.132242,-11.867635,-16.044884,8.140953,-10.125163,9.394515
4,CHUM-008,-0.352161,-0.641335,3.311094,-4.010529,19.108816,-5.374633,5.588564,-3.142635,-0.881599,...,-3.396833,4.848978,-8.309546,13.379717,-5.134465,6.119023,1.801414,-7.728262,5.466892,-7.261157


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2040,2041,2042,2043,2044,2045,2046,2047,Relapse,RFS
0,CHUM-001,4.485706,-1.089238,7.262412,-8.944846,15.662836,-11.749378,9.908555,-8.122204,-5.735908,...,-7.214621,10.840294,-11.032719,11.850606,5.513536,-13.309430,10.613845,-10.545938,0,1704
1,CHUM-002,-0.871721,-2.756554,3.061361,-4.119829,13.619308,-5.428620,3.538110,-1.755272,-1.232317,...,-8.425580,11.245088,-5.558995,5.322546,0.983876,-5.255292,5.934350,-5.383061,1,439
2,CHUM-006,-6.286274,-1.969023,0.316089,4.045898,26.725820,3.302373,2.846397,3.988780,5.453089,...,-12.878074,12.605976,-0.240946,0.288653,-8.028224,-5.678471,1.700145,-1.731813,0,1186
3,CHUM-007,-16.278902,-10.682121,-11.442419,15.026470,26.699356,11.757220,-10.014070,13.172195,14.324217,...,-16.455168,10.636695,12.132242,-11.867635,-16.044884,8.140953,-10.125163,9.394515,0,1702
4,CHUM-008,-0.352161,-0.641335,3.311094,-4.010529,19.108816,-5.374633,5.588564,-3.142635,-0.881599,...,-8.309546,13.379717,-5.134465,6.119023,1.801414,-7.728262,5.466892,-7.261157,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)